***
10 random seeds: range(20, 30)
for data creation for each type of spammer

invoke crowdbt for 10 random seeds: range(10)

get crowdbt accuracy (+- std dev), wacc and tau

save in results/spammer_type/crowdbt.csv
***

## Device Setup

In [1]:
!nvidia-smi

Sun Dec 21 09:31:27 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   54C    P0              47W / 300W |      7MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
print(f"Current PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

Current PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA version: 12.8


### Importing CrowdBT

In [4]:
import sys
sys.path.insert(0, "../")
sys.path.insert(1, "../../")

from spammer_types import *
from util import *
import opt_fair
from distribution_utils import crowd_bt_dist, logistic_preference_dist, comparisons_to_df, safe_kendalltau, to_numpy
from metrics import compute_acc, compute_weighted_acc
from pgem import EMWrapper

## FaceAge dataset

### Get the original df of passage dataset

In [5]:
df_path = "../../real_data/faceage/data/crowd_labels.csv"

In [6]:
import pandas as pd
df = pd.read_csv(df_path)
def sort_df(df, column_name):
        # Sort by a specific column (replace 'column_name' with your column)
        df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

        return df_sorted
df = sort_df(df, 'performer')
df[['left', 'right', 'label', 'performer']].head()

,left,right,label,performer
0,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6306,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6176,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6175,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6174,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0


In [7]:
percents = [10, 20, 40, 60, 80]
# percents = [10]

In [8]:
import pickle

with open("../../real_data/faceage/data/FaceAgeDF.pickle", "rb") as handle:
    df_passage = pickle.load(handle)
df_passage

,full_path,score,gender
0,nm1442940_rm3965098752_1996-10-3_2006.jpg,10,0.0
1,nm4832920_rm1781768448_2003-8-28_2013.jpg,10,0.0
2,nm0652089_rm860657920_1992-3-10_2002.jpg,10,0.0
3,nm0004917_rm1493730304_1969-5-12_1979.jpg,10,0.0
4,nm1113550_rm1332711936_1996-4-14_2006.jpg,10,0.0
...,...,...,...
9145,475367_1941-08-03_2011.jpg,70,1.0
9146,304085_1919-07-07_1989.jpg,70,1.0
9147,nm0001627_rm4164078592_1927-2-20_1997.jpg,70,1.0
9148,nm0000024_rm1715129344_1904-4-14_1974.jpg,70,1.0


In [9]:
size = len(df_passage)
print(size)
classes = [0] * size
# for faceage it would be classes = df_passage['gender']

9150


In [10]:
gt_df = df_passage

### Addition of Random Guessors

In [11]:
spammer_type = "random"

In [12]:
csv_file = f"results/{spammer_type}/crowdbt.csv"

In [13]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [14]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "CrowdBT_acc_mean", "CrowdBT_acc_std",
    "CrowdBT_wacc_mean", "CrowdBT_wacc_std",
    "CrowdBT_tau_mean", "CrowdBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    CrowdBT_accs, CrowdBT_waccs, CrowdBT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_random_spammer(df, percent, seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                crowdbt_test = opt_fair.CrowdBT_3_0(data=PC_faceage, penalty=0, device=device, random_seed=seed)
                crowdbt_scores, _ = crowdbt_test.alternate_optim(size, K)
                r_est = to_numpy(crowdbt_scores)
                if np.isnan(r_est).any():
                    continue
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if CrowdBT_tau < 0:
                    r_est = -r_est
                CrowdBT_acc = compute_acc(gt_df, r_est, device)
                CrowdBT_wacc = compute_weighted_acc(gt_df, r_est, device)
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"CrowdBT failed due to {e}")
                continue
            CrowdBT_accs.append(CrowdBT_acc)
            CrowdBT_waccs.append(CrowdBT_wacc)
            CrowdBT_taus.append(CrowdBT_tau)
    
    row = [
        percent,
        np.mean(CrowdBT_accs), np.std(CrowdBT_accs),
        np.mean(CrowdBT_waccs), np.std(CrowdBT_waccs),
        np.mean(CrowdBT_taus), np.std(CrowdBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"CrowdBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(CrowdBT_accs):.4f} ± {np.std(CrowdBT_accs):.4f} | "
    f"WAcc: {np.mean(CrowdBT_waccs):.4f} ± {np.std(CrowdBT_waccs):.4f} | "
    f"Tau: {np.mean(CrowdBT_taus):.4f} ± {np.std(CrowdBT_taus):.4f}")

Unique performers: 4500
4500


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 334.16it/s, loss=8.76e+4]


Unique performers: 4500
4500


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 364.49it/s, loss=8.75e+4]


Unique performers: 4500
4500


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 429.03it/s, loss=8.76e+4]


Unique performers: 4500
4500


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 391.53it/s, loss=8.75e+4]


Unique performers: 4500
4500


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 365.56it/s, loss=8.76e+4]


### Addition of Anti-Personas

In [ ]:
spammer_type = "anti"

In [ ]:
csv_file = f"results/{spammer_type}/crowdbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "CrowdBT_acc_mean", "CrowdBT_acc_std",
    "CrowdBT_wacc_mean", "CrowdBT_wacc_std",
    "CrowdBT_tau_mean", "CrowdBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    CrowdBT_accs, CrowdBT_waccs, CrowdBT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_anti_personas(df, percent, seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                crowdbt_test = opt_fair.CrowdBT_3_0(data=PC_faceage, penalty=0, device=device, random_seed=seed)
                crowdbt_scores, _ = crowdbt_test.alternate_optim(size, K)
                r_est = to_numpy(crowdbt_scores)
                if np.isnan(r_est).any():
                    continue
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if CrowdBT_tau < 0:
                    r_est = -r_est
                CrowdBT_acc = compute_acc(gt_df, r_est, device)
                CrowdBT_wacc = compute_weighted_acc(gt_df, r_est, device)
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"CrowdBT failed due to {e}")
                continue
            CrowdBT_accs.append(CrowdBT_acc)
            CrowdBT_waccs.append(CrowdBT_wacc)
            CrowdBT_taus.append(CrowdBT_tau)
    
    row = [
        percent,
        np.mean(CrowdBT_accs), np.std(CrowdBT_accs),
        np.mean(CrowdBT_waccs), np.std(CrowdBT_waccs),
        np.mean(CrowdBT_taus), np.std(CrowdBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"CrowdBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(CrowdBT_accs):.4f} ± {np.std(CrowdBT_accs):.4f} | "
    f"WAcc: {np.mean(CrowdBT_waccs):.4f} ± {np.std(CrowdBT_waccs):.4f} | "
    f"Tau: {np.mean(CrowdBT_taus):.4f} ± {np.std(CrowdBT_taus):.4f}")

### Addition of Left Position biased Spammers

In [ ]:
spammer_type = "left"

In [ ]:
csv_file = f"results/{spammer_type}/crowdbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "CrowdBT_acc_mean", "CrowdBT_acc_std",
    "CrowdBT_wacc_mean", "CrowdBT_wacc_std",
    "CrowdBT_tau_mean", "CrowdBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    CrowdBT_accs, CrowdBT_waccs, CrowdBT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="left", seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                crowdbt_test = opt_fair.CrowdBT_3_0(data=PC_faceage, penalty=0, device=device, random_seed=seed)
                crowdbt_scores, _ = crowdbt_test.alternate_optim(size, K)
                r_est = to_numpy(crowdbt_scores)
                if np.isnan(r_est).any():
                    continue
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if CrowdBT_tau < 0:
                    r_est = -r_est
                CrowdBT_acc = compute_acc(gt_df, r_est, device)
                CrowdBT_wacc = compute_weighted_acc(gt_df, r_est, device)
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"CrowdBT failed due to {e}")
                continue
            CrowdBT_accs.append(CrowdBT_acc)
            CrowdBT_waccs.append(CrowdBT_wacc)
            CrowdBT_taus.append(CrowdBT_tau)
    
    row = [
        percent,
        np.mean(CrowdBT_accs), np.std(CrowdBT_accs),
        np.mean(CrowdBT_waccs), np.std(CrowdBT_waccs),
        np.mean(CrowdBT_taus), np.std(CrowdBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"CrowdBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(CrowdBT_accs):.4f} ± {np.std(CrowdBT_accs):.4f} | "
    f"WAcc: {np.mean(CrowdBT_waccs):.4f} ± {np.std(CrowdBT_waccs):.4f} | "
    f"Tau: {np.mean(CrowdBT_taus):.4f} ± {np.std(CrowdBT_taus):.4f}")

### Addition of Right Position biased Spammers

In [ ]:
spammer_type = "right"

In [ ]:
csv_file = f"results/{spammer_type}/crowdbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "CrowdBT_acc_mean", "CrowdBT_acc_std",
    "CrowdBT_wacc_mean", "CrowdBT_wacc_std",
    "CrowdBT_tau_mean", "CrowdBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    CrowdBT_accs, CrowdBT_waccs, CrowdBT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="right", seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                crowdbt_test = opt_fair.CrowdBT_3_0(data=PC_faceage, penalty=0, device=device, random_seed=seed)
                crowdbt_scores, _ = crowdbt_test.alternate_optim(size, K)
                r_est = to_numpy(crowdbt_scores)
                if np.isnan(r_est).any():
                    continue
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if CrowdBT_tau < 0:
                    r_est = -r_est
                CrowdBT_acc = compute_acc(gt_df, r_est, device)
                CrowdBT_wacc = compute_weighted_acc(gt_df, r_est, device)
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"CrowdBT failed due to {e}")
                continue
            CrowdBT_accs.append(CrowdBT_acc)
            CrowdBT_waccs.append(CrowdBT_wacc)
            CrowdBT_taus.append(CrowdBT_tau)
    
    row = [
        percent,
        np.mean(CrowdBT_accs), np.std(CrowdBT_accs),
        np.mean(CrowdBT_waccs), np.std(CrowdBT_waccs),
        np.mean(CrowdBT_taus), np.std(CrowdBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"CrowdBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(CrowdBT_accs):.4f} ± {np.std(CrowdBT_accs):.4f} | "
    f"WAcc: {np.mean(CrowdBT_waccs):.4f} ± {np.std(CrowdBT_waccs):.4f} | "
    f"Tau: {np.mean(CrowdBT_taus):.4f} ± {np.std(CrowdBT_taus):.4f}")

### Addition of Equal proportion of all four kind of spammers

In [ ]:
spammer_type = "equal"

In [ ]:
csv_file = f"results/{spammer_type}/crowdbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "CrowdBT_acc_mean", "CrowdBT_acc_std",
    "CrowdBT_wacc_mean", "CrowdBT_wacc_std",
    "CrowdBT_tau_mean", "CrowdBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    CrowdBT_accs, CrowdBT_waccs, CrowdBT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_equal_proportion_of_all_spammers(df, percent, seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                crowdbt_test = opt_fair.CrowdBT_3_0(data=PC_faceage, penalty=0, device=device, random_seed=seed)
                crowdbt_scores, _ = crowdbt_test.alternate_optim(size, K)
                r_est = to_numpy(crowdbt_scores)
                if np.isnan(r_est).any():
                    continue
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if CrowdBT_tau < 0:
                    r_est = -r_est
                CrowdBT_acc = compute_acc(gt_df, r_est, device)
                CrowdBT_wacc = compute_weighted_acc(gt_df, r_est, device)
                CrowdBT_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"CrowdBT failed due to {e}")
                continue
            CrowdBT_accs.append(CrowdBT_acc)
            CrowdBT_waccs.append(CrowdBT_wacc)
            CrowdBT_taus.append(CrowdBT_tau)
    
    row = [
        percent,
        np.mean(CrowdBT_accs), np.std(CrowdBT_accs),
        np.mean(CrowdBT_waccs), np.std(CrowdBT_waccs),
        np.mean(CrowdBT_taus), np.std(CrowdBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"CrowdBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(CrowdBT_accs):.4f} ± {np.std(CrowdBT_accs):.4f} | "
    f"WAcc: {np.mean(CrowdBT_waccs):.4f} ± {np.std(CrowdBT_waccs):.4f} | "
    f"Tau: {np.mean(CrowdBT_taus):.4f} ± {np.std(CrowdBT_taus):.4f}")